In [70]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [92]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
musinsa_list = []

In [93]:
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
try:
    driver.maximize_window() # 브라우저의 크기 전체화면으로 확대
except:
    pass

In [94]:
interval = 1 # 1초에 한번씩 스크롤 내림

 # 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
while True:
        # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
    time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height

In [95]:
columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
values = []   
soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

In [100]:
for i, row in enumerate(data_rows): 
    print('{}번째 코디 크롤링'.format(i+1))
    blank=[]

    codimap_category = row.find ('span', attrs= {'class':'style-list-information__text'})
    if codimap_category:
        codimap_category=codimap_category.get_text().strip()
        blank.append(codimap_category)
    else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
        blank.append('Something is wrong')
        print('{}번째 코디 카테고리 가져올때 문제발생'.format(i+1))
        continue

    codimap_title = row.find('strong', attrs= {'class':'style-list-information__title'})
    if codimap_title:
        codimap_title=codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디 제목 가져올때 문제 발생'.format(i+1))
        continue
    
    codimap_date= row.find('span', attrs= {'class': 'post-information__text'})
    if codimap_date:
        codimap_date=codimap_date.get_text().strip()
        blank.append(codimap_date)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디 날짜 가져올때 문제 발생'.format(i+1))
        continue

    views= row.find_all('span', attrs= {'class': 'post-information__text'})[1]
    if views:
        text=views.get_text().strip()
        views = text.split()[1]
        blank.append(str(views))
    else:
        blank.append('Something is wrong')
        print('{}번째 조회수 가져올때 문제 발생'.format(i+1))
        continue
    
    comment_numbers= row.find_all('span', attrs= {'class': 'post-information__text'})[2]
    if views:
        text=comment_numbers.get_text().strip()
        comment_numbers = text.split()[1]
        blank.append(str(comment_numbers))
    else:
        blank.append('Something is wrong')
        print('{}번째 댓글 가져올때 문제 발생'.format(i+1))
        continue


    values.append(blank)
    print('---------------------------------------------------')
df = pd.DataFrame(values, columns = columns)
musinsa_list.append(df)

1번째 코디 크롤링
---------------------------------------------------
2번째 코디 크롤링
---------------------------------------------------
3번째 코디 크롤링
---------------------------------------------------
4번째 코디 크롤링
---------------------------------------------------
5번째 코디 크롤링
---------------------------------------------------
6번째 코디 크롤링
---------------------------------------------------
7번째 코디 크롤링
---------------------------------------------------
8번째 코디 크롤링
---------------------------------------------------
9번째 코디 크롤링
---------------------------------------------------
10번째 코디 크롤링
---------------------------------------------------
11번째 코디 크롤링
---------------------------------------------------
12번째 코디 크롤링
---------------------------------------------------
13번째 코디 크롤링
---------------------------------------------------
14번째 코디 크롤링
---------------------------------------------------
15번째 코디 크롤링
---------------------------------------------------
16번째 코디 크롤링
-------------------------------------

In [103]:
df_ = pd.concat(musinsa_list).reset_index(drop=True)
df_

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"16,085",49
1,캐주얼,블랙 매니아,21.02.23,"34,675",46
2,캐주얼,오늘의 코디,22.03.22,"12,201",38
3,댄디,추천 댄디 룩,21.09.15,"20,187",27
4,캐주얼,릴랙스 룩,21.02.23,"23,539",22
5,스트릿,힙하다 힙해,21.02.03,"35,289",21
6,캐주얼,심플한 코디,21.07.30,"39,584",18
7,댄디,데이트 코디,21.12.24,"10,312",18
8,댄디,훈훈 남친 룩,21.03.24,"19,346",16
9,캐주얼,지금 입기 좋아,21.08.31,"20,215",16


In [104]:
df.to_csv('//Users/yuna/Desktop/git real/ybigta-assignment/김서윤_크롤링과제.csv', encoding='utf-8-sig', index=False)